In [1]:
import os
import shutil
import random

# Directorio donde están las imágenes originales
original_dir = r"C:\WorkPython\Ciencia De Datos\Datos\ojos_malitos"

# Directorios de destino
train_dir = "dataset/train"
test_dir = "dataset/test"

# Porcentaje de datos para entrenamiento
split_ratio = 0.8

# Crear carpetas de train y test si no existen
for folder in os.listdir(original_dir):
    os.makedirs(os.path.join(train_dir, folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir, folder), exist_ok=True)

    # Listar imágenes de la carpeta actual
    images = os.listdir(os.path.join(original_dir, folder))
    random.shuffle(images)

    # Dividir imágenes en train y test
    split_idx = int(len(images) * split_ratio)
    train_images = images[:split_idx]
    test_images = images[split_idx:]

    # Copiar imágenes a sus respectivas carpetas
    for img in train_images:
        shutil.copy(os.path.join(original_dir, folder, img), os.path.join(train_dir, folder, img))
    
    for img in test_images:
        shutil.copy(os.path.join(original_dir, folder, img), os.path.join(test_dir, folder, img))

print("División completada. Train y Test creados en 'dataset/'.")

División completada. Train y Test creados en 'dataset/'.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224  # Tamaño estándar

train_datagen = ImageDataGenerator(
    rescale=1./255,        # Normaliza los valores entre 0 y 1
    rotation_range=20,     # Rota aleatoriamente la imagen
    width_shift_range=0.2, # Desplaza la imagen horizontalmente
    height_shift_range=0.2,# Desplaza la imagen verticalmente
    shear_range=0.2,       # Aplica transformaciones angulares
    zoom_range=0.2,        # Hace zoom sobre la imagen
    horizontal_flip=True,  # Invierte horizontalmente la imagen
    fill_mode='nearest'    
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Solo normalización para test

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical'  # Usamos 'categorical' porque tenemos 4 clases (one-hot encoding)
)

test_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical'
)

Found 3372 images belonging to 4 classes.
Found 845 images belonging to 4 classes.


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  # Reduce sobreajuste
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  # Reduce sobreajuste
    Dense(4, activation='softmax')  # 4 clases → Softmax
])

# Compilación
model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     4,718,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,118,340 (19.52 MB)

 Trainable params: 5,116,996 (19.52 MB)

 Non-trainable params: 1,344 (5.25 KB)

In [9]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20,
    verbose=1
)

Epoch 1/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 545s 4s/step - categorical_accuracy: 0.3689 - loss: 1.8620 - val_categorical_accuracy: 0.2604 - val_loss: 3.6239
Epoch 2/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 306s 3s/step - categorical_accuracy: 0.3768 - loss: 1.6173 - val_categorical_accuracy: 0.2604 - val_loss: 2.4461
Epoch 3/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - categorical_accuracy: 0.4266 - loss: 1.3698 - val_categorical_accuracy: 0.2604 - val_loss: 4.7379
Epoch 4/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 221s 2s/step - categorical_accuracy: 0.4553 - loss: 1.2501 - val_categorical_accuracy: 0.2734 - val_loss: 2.9571
Epoch 5/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - categorical_accuracy: 0.4984 - loss: 1.1604 - val_categorical_accuracy: 0.2615 - val_loss: 4.1370
Epoch 6/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - categorical_accuracy: 0.5257 - loss: 1.1051 - val_categorical_accuracy: 0.5491 - val_loss: 1.0521
Epoch 7/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - categorical_accuracy: 0.571

In [10]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Accuracy en test: {test_acc:.4f}')

27/27 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - categorical_accuracy: 0.7323 - loss: 0.7122
Accuracy en test: 0.7041
